In [1]:
import dionysus as d
import numpy as np
from itertools import combinations

# Problem 1

\begin{equation}
H(x, t) = (1-t) x + t x / \|x\|_2
\end{equation}

Is pretty easy/straightforward.  This is not unique, and there are other possibilities.

# Problem 2


In [2]:
from scipy.spatial.distance import pdist, squareform

theta = np.linspace(0, 2*np.pi, 3, endpoint=False)
X = np.c_[np.cos(theta), np.sin(theta)]
squareform(pdist(X))

array([[0.        , 1.73205081, 1.73205081],
       [1.73205081, 0.        , 1.73205081],
       [1.73205081, 1.73205081, 0.        ]])

In [3]:
np.sqrt(3)/2

0.8660254037844386

a) There are two regimes where this occurs.  First, when both complexes are just three points, and second when the two complexes are the two simplex $(0,1,2)$ (plus faces).  The three points are equidistant, with $d(x,y) = \sqrt{3}$

$R(X;r)$ is three points for $r < \sqrt{3}$, then equal to the 2-simplex for $r\ge \sqrt(3)$.

$C(X;r)$ is three proints for $r < \sqrt{3}/2$, then equal to the 2-simplex boundary (3 edges) for $\sqrt{3}/2 \le r < 1$ (the balls first have a 3-way intersection at the origin, distance 1 away), then equal to the 2-simplex for $r\ge 1$.

Thus the two complexes are identical for $r\in [0,\sqrt{3}/2) \cup [\sqrt{3},\infty)$

b) You don't need to construct an explicit homotopy.  $C(X;r) \simeq S^1$ when it is identical to the 2-simplex boundary without the 2-simplex.  This is  in the range $\sqrt{3}/2 \le r < 1$

c) $R(X; r)$ is never $\simeq S^1$.

# Rips Complex Construction

Here's a solution using Dionysus to sore the simplicial complexes.  You actually just use a `Filtration` type, but don't specify any filtration parameters.

In [4]:
def RipsComplex(X, r, k):
    """
    Construct Rips complex on point cloud X with parameter r, max simplex dimension k
    """
    n = len(X)
    
    def in_rips(s):
        """
        determine if simplex s is in Rips complex
        """
        for i,j in combinations(s, 2):
            if np.linalg.norm(X[i] - X[j]) > r:
                return False
            
        return True
    
    R = d.Filtration()
    
    # all vertices in R
    for i in range(n):
        R.append(d.Simplex((i,)))
        
    # add higher dimensional simplices if appropriate
    for dim in range(1,k+1):
        for s in combinations(range(n), dim+1):
            if in_rips(s):
                R.append(d.Simplex(s))
        
    return R

In [5]:
X = np.random.randn(10,3)
R = RipsComplex(X, 1.0, 2)
for s in R:
    print(s)

<0> 0
<1> 0
<2> 0
<3> 0
<4> 0
<5> 0
<6> 0
<7> 0
<8> 0
<9> 0
<1,6> 0
<1,8> 0
<4,9> 0


# Nerve Construction

In [6]:
def Nerve(C, k):
    """
    return Nerve of a cover with maximum simplex dimension k
    Cover is a list of sets
    """
    n = len(C)
    
    def cover_intersection(s):
        """
        return intersection of sets in s
        """
        return set.intersection(*tuple(C[i] for i in s))
        
    N = d.Filtration()
    
        
    # add simplices if appropriate
    for dim in range(k+1):
        for s in combinations(range(n), dim+1):
            if cover_intersection(s):
                N.append(d.Simplex(s))
        
    return N
    

In [7]:
C = [{0,1, 2}, {1,2}, {0,2}, {0,3}]
N = Nerve(C, 2)
for s in N:
    print(s)

<0> 0
<1> 0
<2> 0
<3> 0
<0,1> 0
<0,2> 0
<0,3> 0
<1,2> 0
<2,3> 0
<0,1,2> 0
<0,2,3> 0


# Problem 3

Note that in this problem, you don't need to use the same parameters when constructing the mapper graph.  Just load the data and construct _some_ mapper graph.

In [8]:
# Import the class
import kmapper as km
import sklearn.manifold as manifold

# Some sample data
from sklearn import datasets

import pandas as pd

In [9]:
df = pd.read_csv("https://raw.githubusercontent.com/stat37411/tda/main/data/chemdiab.csv")
df2 = df[["rw", "fpg", "ga", "ina", "sspg", "cc"]]

In [10]:
df

,rw,fpg,ga,ina,sspg,cc
0,0.81,80,356,124,55,Normal
1,0.95,97,289,117,76,Normal
2,0.94,105,319,143,105,Normal
3,1.04,90,356,199,108,Normal
4,1.00,90,323,240,143,Normal
...,...,...,...,...,...,...
140,1.05,353,1428,41,480,Overt_Diabetic
141,0.91,180,923,77,150,Overt_Diabetic
142,0.90,213,1025,29,209,Overt_Diabetic
143,1.11,328,1246,124,442,Overt_Diabetic


In [11]:
data = df2[['rw', 'fpg', 'ga', 'ina', 'sspg']].to_numpy()
labels = df2[['cc']].to_numpy()
data.shape

(145, 5)

In [12]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KernelDensity
from sklearn.cluster import DBSCAN

In [13]:
KernelDensity().fit(data)

KernelDensity()

In [14]:
# Initialize
mapper = km.KeplerMapper(verbose=1)

# Fit to and transform the data
projected_data = mapper.fit_transform(data, 
                                      #projection=manifold.Isomap(n_components=100, n_jobs=-1)
                                      #projection=[1,0,0,0,0]
                                      projection=PCA(n_components=2)
                                      #projection=KernelDensity()
                                     ) # X-Y axis

# Create dictionary called 'graph' with nodes, edges and meta-information
graph = mapper.map(projected_data, data,
                  clusterer=DBSCAN(eps=200, min_samples=2),
                  cover=km.Cover(n_cubes=20, perc_overlap=0.6))

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: PCA(n_components=2)
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (145, 5)

..Projecting data using: 
	PCA(n_components=2)


..Scaling with: MinMaxScaler()

Mapping on data shaped (145, 5) using lens shaped (145, 2)

Creating 400 hypercubes.

Created 619 edges and 122 nodes in 0:00:00.094224.


In [15]:
# Visualize it
mapper.visualize(graph, path_html="chemdiab_keplermapper_output.html",
                 title="make_circles(n_samples=5000, noise=0.03, factor=0.3)")

from kmapper import jupyter # Creates custom CSS full-size Jupyter screen

# Inline display
# jupyter.display(path_html="http://mlwave.github.io/tda/word2vec-gender-bias.html")
jupyter.display(path_html="chemdiab_keplermapper_output.html")

Wrote visualization to: chemdiab_keplermapper_output.html


/home/brad/miniconda3/envs/tda/lib/python3.9/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
